# Data Cleaning Notebook

In [247]:
import pandas as pd
import os

## Creating Compressed Data

In [248]:
# # gzip file creation block
# root_dir = "/Users/trustanprice/Desktop/Personal/Basketball-Predictions/data/raw/team-stats"

# for subdir, _, files in os.walk(root_dir):
#     for file in files:
#         if file.endswith(".csv"):
#             file_path = os.path.join(subdir, file)
#             gz_path = file_path + ".gz"

#             try:
#                 df = pd.read_csv(file_path)

#                 if df.empty:
#                     print(f"Skipping empty file: {file_path}")
#                     continue

#                 print(f"Compressing {file_path} -> {gz_path}")
#                 df.to_csv(gz_path, index=False, compression="gzip")

#             except pd.errors.EmptyDataError:
#                 print(f"Skipping empty file (no data): {file_path}")


## CSV Cleaning

For each csv file, I have already gone in and manually lightly cleaned the data with Microsoft Excel; however, for deeper cleaning I will use python libraries here in this ipynb file.

In Microsoft Excel, I:
- eliminated unnecessary rows and columns from the dataset
- added in the columns necessary for aggregation and merging
- formatted everything correctly to prepare it for python data manipulation

In [249]:
team_records_df = pd.read_csv("/Users/trustanprice/Desktop/Personal/Basketball-Predictions/data/raw/team-stats/team-records.csv")
team_records_df.head(5)

,Rk,Team,Home,Road,E,W,Pre-ASG,Post-ASG,Season
0,1,Oklahoma City Thunder,36-6,32-8,29-1,39-13,44-10,24-4,2025
1,2,Cleveland Cavaliers,34-7,30-11,41-11,23-7,44-10,20-8,2025
2,3,Boston Celtics,28-13,33-8,39-13,22-8,39-16,22-5,2025
3,4,Houston Rockets,29-12,23-18,21-9,31-21,34-21,18-9,2025
4,5,New York Knicks,27-14,24-17,34-18,17-13,36-18,15-13,2025


For this team_records_df, I need to make it so that the wins and loss columns are seperated.

In [250]:
# Columns to split
split_cols = ["Home", "Road", "E", "W", "Pre-ASG", "Post-ASG"]

# Loop through and split each column into wins/losses
for col in split_cols:
    team_records_df[[f"{col}_W", f"{col}_L"]] = team_records_df[col].str.split("-", expand=True).astype(int)

# remove original columns
team_records_df = team_records_df.drop(columns=split_cols)

# Check results
team_records_df.head()


,Rk,Team,Season,Home_W,Home_L,Road_W,Road_L,E_W,E_L,W_W,W_L,Pre-ASG_W,Pre-ASG_L,Post-ASG_W,Post-ASG_L
0,1,Oklahoma City Thunder,2025,36,6,32,8,29,1,39,13,44,10,24,4
1,2,Cleveland Cavaliers,2025,34,7,30,11,41,11,23,7,44,10,20,8
2,3,Boston Celtics,2025,28,13,33,8,39,13,22,8,39,16,22,5
3,4,Houston Rockets,2025,29,12,23,18,21,9,31,21,34,21,18,9
4,5,New York Knicks,2025,27,14,24,17,34,18,17,13,36,18,15,13


In [251]:
team_stats_df = pd.read_csv("/Users/trustanprice/Desktop/Personal/Basketball-Predictions/data/raw/team-stats/team-stats.csv")
team_stats_df.head().shape

(5, 28)

### Optimal Dataframe Construction

I now want to merge the dataframes that contain the team records and team stats (FG%, FGA, MIN) position. I will do this by merging by team and season.

In [252]:
# Merge on Team and Season
team_df = pd.merge(
    team_stats_df,
    team_records_df,
    on=["Team", "Season"],
    how="inner"  
)

# Check result
print(team_df.head())
print(f"-----")
print(f"The shape of the dataframe:", team_df.shape)

   Season                   Team  GP   W   L   WIN%   Min    PTS   FGM   FGA  \
0    2025  Oklahoma City Thunder  82  68  14  0.829  48.1  120.5  44.6  92.7   
1    2025    Cleveland Cavaliers  82  64  18  0.780  48.2  121.9  44.5  90.8   
2    2025         Boston Celtics  82  61  21  0.744  48.4  116.3  41.6  90.0   
3    2025        Houston Rockets  82  52  30  0.634  48.2  114.3  42.5  93.4   
4    2025        New York Knicks  82  51  31  0.622  48.5  115.8  43.3  89.2   

   ...  Road_W  Road_L  E_W  E_L  W_W  W_L  Pre-ASG_W  Pre-ASG_L  Post-ASG_W  \
0  ...      32       8   29    1   39   13         44         10          24   
1  ...      30      11   41   11   23    7         44         10          20   
2  ...      33       8   39   13   22    8         39         16          22   
3  ...      23      18   21    9   31   21         34         21          18   
4  ...      24      17   34   18   17   13         36         18          15   

   Post-ASG_L  
0           4  
1     

In [253]:
draft_df = pd.read_csv("/Users/trustanprice/Desktop/Personal/Basketball-Predictions/data/raw/team-stats/draft.csv")
draft_df.head()

,Pk,Tm,Season
0,1,DAL,2025
1,2,SAS,2025
2,3,PHI,2025
3,4,CHO,2025
4,5,UTA,2025


In [254]:
coach_df = pd.read_csv("/Users/trustanprice/Desktop/Personal/Basketball-Predictions/data/raw/team-stats/coach.csv")
coach_df.head(5)

,Coach,Tm,Yw/Franch,YOverall,CareerW,CareerL,CareerW%,Season
0,Quin Snyder,ATL,3,11,458,363,0.558,2025
1,Joe Mazzulla,BOS,3,3,182,64,0.740,2025
2,Jordi Fernandez,BRK,1,1,26,56,0.317,2025
3,Billy Donovan,CHI,5,10,438,362,0.548,2025
4,Charles Lee,CHO,1,1,19,63,0.232,2025


In [255]:
# Map of acronyms to full team names
team_map = {
    "ATL": "Atlanta Hawks",
    "BOS": "Boston Celtics",
    "BRK": "Brooklyn Nets",  # sometimes shown as BRK or BKN
    "BKN": "Brooklyn Nets",
    "CHI": "Chicago Bulls",
    "CHO": "Charlotte Hornets",  # CHA/CHO both used historically
    "CHA": "Charlotte Hornets",
    "CLE": "Cleveland Cavaliers",
    "DAL": "Dallas Mavericks",
    "DEN": "Denver Nuggets",
    "DET": "Detroit Pistons",
    "GSW": "Golden State Warriors",
    "HOU": "Houston Rockets",
    "IND": "Indiana Pacers",
    "LAC": "Los Angeles Clippers",
    "LAL": "Los Angeles Lakers",
    "MEM": "Memphis Grizzlies",
    "MIA": "Miami Heat",
    "MIL": "Milwaukee Bucks",
    "MIN": "Minnesota Timberwolves",
    "NOP": "New Orleans Pelicans",
    "NYK": "New York Knicks",
    "OKC": "Oklahoma City Thunder",
    "ORL": "Orlando Magic",
    "PHI": "Philadelphia 76ers",
    "PHO": "Phoenix Suns",
    "POR": "Portland Trail Blazers",
    "SAC": "Sacramento Kings",
    "SAS": "San Antonio Spurs",
    "TOR": "Toronto Raptors",
    "UTA": "Utah Jazz",
    "WAS": "Washington Wizards"
}

# Replace acronyms with full names
coach_df["Team"] = coach_df["Tm"].map(team_map)
draft_df["Team"] = draft_df["Tm"].map(team_map)

# Drop the old acronym column
coach_df = coach_df.drop(columns=["Tm"])
draft_df = draft_df.drop(columns=["Tm"])

# Check result
print(coach_df.head())
print(f"The shape of the coaches df:", coach_df.shape)
print(f"----------------------------------------------")
print(draft_df.head())
print(f"The shape of the draft df:", draft_df.shape)


             Coach  Yw/Franch  YOverall  CareerW  CareerL  CareerW%  Season  \
0      Quin Snyder          3        11      458      363     0.558    2025   
1     Joe Mazzulla          3         3      182       64     0.740    2025   
2  Jordi Fernandez          1         1       26       56     0.317    2025   
3    Billy Donovan          5        10      438      362     0.548    2025   
4      Charles Lee          1         1       19       63     0.232    2025   

                Team  
0      Atlanta Hawks  
1     Boston Celtics  
2      Brooklyn Nets  
3      Chicago Bulls  
4  Charlotte Hornets  
The shape of the coaches df: (328, 8)
----------------------------------------------
   Pk  Season                Team
0   1    2025    Dallas Mavericks
1   2    2025   San Antonio Spurs
2   3    2025  Philadelphia 76ers
3   4    2025   Charlotte Hornets
4   5    2025           Utah Jazz
The shape of the draft df: (659, 3)


I now want to merge the dataframes that contain the length of the current coach and their draft position. I will do this by merging by team and season.

In [256]:
# Merge on Team and Season
front_office_df = pd.merge(
    coach_df,
    draft_df,
    on=["Team", "Season"],
    how="inner"  
)

print(front_office_df.head())
print(f"-----")
print(f"The shape of the dataframe:", front_office_df.shape)

             Coach  Yw/Franch  YOverall  CareerW  CareerL  CareerW%  Season  \
0      Quin Snyder          3        11      458      363     0.558    2025   
1      Quin Snyder          3        11      458      363     0.558    2025   
2     Joe Mazzulla          3         3      182       64     0.740    2025   
3     Joe Mazzulla          3         3      182       64     0.740    2025   
4  Jordi Fernandez          1         1       26       56     0.317    2025   

             Team  Pk  
0   Atlanta Hawks  13  
1   Atlanta Hawks  22  
2  Boston Celtics  28  
3  Boston Celtics  32  
4   Brooklyn Nets   8  
-----
The shape of the dataframe: (648, 9)


I now need to add a column to this front offfice dataframe that counts the amount of coaches each team has in each season.

In [257]:
# Count how many distinct coaches each Team had in each Season
coach_counts = coach_df.groupby(["Team", "Season"])["Coach"].nunique().reset_index()
coach_counts.rename(columns={"Coach": "Coach_Count"}, inplace=True)

# Merge that count into your front_office_df
front_office_df = pd.merge(
    front_office_df,
    coach_counts,
    on=["Team", "Season"],
    how="left"
)

print(front_office_df.head())
print(f"-----")
print(f"The shape of the dataframe:", front_office_df.shape)

             Coach  Yw/Franch  YOverall  CareerW  CareerL  CareerW%  Season  \
0      Quin Snyder          3        11      458      363     0.558    2025   
1      Quin Snyder          3        11      458      363     0.558    2025   
2     Joe Mazzulla          3         3      182       64     0.740    2025   
3     Joe Mazzulla          3         3      182       64     0.740    2025   
4  Jordi Fernandez          1         1       26       56     0.317    2025   

             Team  Pk  Coach_Count  
0   Atlanta Hawks  13            1  
1   Atlanta Hawks  22            1  
2  Boston Celtics  28            1  
3  Boston Celtics  32            1  
4   Brooklyn Nets   8            1  
-----
The shape of the dataframe: (648, 10)


In [258]:
front_office_df[front_office_df["Coach_Count"] > 1]

,Coach,Yw/Franch,YOverall,CareerW,CareerL,CareerW%,Season,Team,Pk,Coach_Count
26,Taylor Jenkins,6,6,250,214,0.539,2025,Memphis Grizzlies,16,2
27,Taylor Jenkins,6,6,250,214,0.539,2025,Memphis Grizzlies,48,2
28,Taylor Jenkins,6,6,250,214,0.539,2025,Memphis Grizzlies,56,2
29,Tuomas Iisalo,1,1,4,5,0.444,2025,Memphis Grizzlies,16,2
30,Tuomas Iisalo,1,1,4,5,0.444,2025,Memphis Grizzlies,48,2
...,...,...,...,...,...,...,...,...,...,...
634,Jeff Hornacek,3,3,101,112,0.474,2016,Phoenix Suns,34,2
635,Earl Watson,1,1,9,24,0.273,2016,Phoenix Suns,4,2
636,Earl Watson,1,1,9,24,0.273,2016,Phoenix Suns,13,2
637,Earl Watson,1,1,9,24,0.273,2016,Phoenix Suns,28,2


I will now add the teams payroll data into the front office dataframe. This will just give my front office dataset that last bit of kick to make accurate predictions on the future of NBA Teams!

In [259]:
team_payroll_df = pd.read_csv("/Users/trustanprice/Desktop/Personal/Basketball-Predictions/data/raw/team-stats/team-payroll.csv")
print(team_payroll_df.head())

                Team      2016       2017       2018       2019       2020  \
0      Atlanta Hawks  71661760   96315163   99992696   79180081  110702618   
1     Boston Celtics  77141919   93465326  115284776  125334993  117759332   
2      Brooklyn Nets  80258302   82391482   95475397  118850600  118889943   
3  Charlotte Hornets  76860006  103054004  117382664  121427859   96552033   
4      Chicago Bulls  87073838   92522306   90466801  112598201  112601901   

        2021       2022       2023       2024       2025       2026  \
0  118804016  135166020  149836313  159153393  170057021  190318494   
1  132931565  136557646  178633307  186940921  195348491  204311991   
2  170444633  174811922  159566723  155015136  168312896  139805782   
3  108218809  122139566  125874047  140774484  168575524  180776048   
4  128963580  136083814  151964990  165630436  165722496  176052328   

        2027       2028         2029  
0  120120521   62041800   45919890.0  
1  170903314  176353230  1

In [260]:
# Reshape payroll_df from wide to long
payroll_long = team_payroll_df.melt(
    id_vars=["Team"],
    var_name="Season",
    value_name="Payroll"
)

# Convert Season column to int
payroll_long["Season"] = payroll_long["Season"].astype(int)

# Merge with front_office_df on Team and Season
front_office_df = pd.merge(
    front_office_df,
    payroll_long,
    on=["Team", "Season"],
    how="left"
)

print(front_office_df.head())
print(f"Shape: {front_office_df.shape}")

             Coach  Yw/Franch  YOverall  CareerW  CareerL  CareerW%  Season  \
0      Quin Snyder          3        11      458      363     0.558    2025   
1      Quin Snyder          3        11      458      363     0.558    2025   
2     Joe Mazzulla          3         3      182       64     0.740    2025   
3     Joe Mazzulla          3         3      182       64     0.740    2025   
4  Jordi Fernandez          1         1       26       56     0.317    2025   

             Team  Pk  Coach_Count      Payroll  
0   Atlanta Hawks  13            1  170057021.0  
1   Atlanta Hawks  22            1  170057021.0  
2  Boston Celtics  28            1  195348491.0  
3  Boston Celtics  32            1  195348491.0  
4   Brooklyn Nets   8            1  168312896.0  
Shape: (648, 11)


### Strength of Schedule Calculation

Next, I need to use the dataframe containing the teams records against themselves to calculate each teams strength of schedule, giving me an even more accurate prediction of the amount of wins they will have in the 2025-26 season.

The formula I will use for the calculation is:
∑(Opposing Team Win Pct * Games vs the Opponent) / Total Games


In [261]:
team_sos_df = pd.read_csv("/Users/trustanprice/Desktop/Personal/Basketball-Predictions/data/raw/team-stats/team-sos.csv")
team_sos_df = team_sos_df.rename(columns=team_map)

print(team_sos_df.head())

   Rk               Team Atlanta Hawks Boston Celtics Brooklyn Nets  \
0   1      Atlanta Hawks           NaN            2-1           2-1   
1   2     Boston Celtics           1-2            NaN           4-0   
2   3      Brooklyn Nets           1-2            0-4           NaN   
3   4      Chicago Bulls           2-2            1-3           2-1   
4   5  Charlotte Hornets           0-4            0-4           1-3   

  Chicago Bulls Charlotte Hornets Cleveland Cavaliers Dallas Mavericks  \
0           2-2               4-0                 2-1              0-2   
1           3-1               4-0                 2-2              1-1   
2           1-2               3-1                 0-4              1-1   
3           NaN               3-1                 0-4              0-2   
4           1-3               NaN                 0-4              1-1   

  Denver Nuggets  ... Orlando Magic Philadelphia 76ers Phoenix Suns  \
0            0-2  ...           2-2                3-0   

In [262]:
# Build mapping (Team+Season → WinPct) from team_df
opp_winpct = dict(zip(zip(team_df["Team"], team_df["Season"]), team_df["WIN%"]))

# Calculate Strength of Schedule
sos_list = []

for _, row in team_sos_df.iterrows():
    team = row["Team"]
    season = row["Season"]

    total_weighted = 0
    total_games = 0

    for opp, record in row.items():
        if opp in ["Rk", "Team", "Season"] or pd.isna(record):
            continue

        try:
            wins, losses = map(int, str(record).split("-"))
        except:
            continue

        games = wins + losses
        opp_team = opp
        key = (opp_team, season)

        if key in opp_winpct:
            total_weighted += opp_winpct[key] * games
            total_games += games

    sos = total_weighted / total_games if total_games > 0 else None
    sos_list.append([team, season, sos])

# --- Step 3: Build SOS DataFrame ---
sos_df = pd.DataFrame(sos_list, columns=["Team", "Season", "SOS"])

print(sos_df.head())
print(f"Shape: {sos_df.shape}")

                Team  Season       SOS
0      Atlanta Hawks    2025  0.492137
1     Boston Celtics    2025  0.475838
2      Brooklyn Nets    2025  0.504637
3      Chicago Bulls    2025  0.492900
4  Charlotte Hornets    2025  0.501437
Shape: (300, 3)


Now that I have calculated the strength of schedule for each team in each season, I will merge the strength of schedule dataframe to the current team_df. This will make it so that I have a front_office_df that includes payroll, draft order, and coaches along with a team_df that will contain team records, strength of schedule, and average stats.

In [263]:
# Merge on SOS and current team_df
team_df = pd.merge(
    team_df,
    sos_df,
    on=["Team", "Season"],
    how="inner"  
)

# Check result
print(team_df.head())
print(f"-----")
print(f"The shape of the dataframe:", team_df.shape)

   Season                   Team  GP   W   L   WIN%   Min    PTS   FGM   FGA  \
0    2025  Oklahoma City Thunder  82  68  14  0.829  48.1  120.5  44.6  92.7   
1    2025    Cleveland Cavaliers  82  64  18  0.780  48.2  121.9  44.5  90.8   
2    2025         Boston Celtics  82  61  21  0.744  48.4  116.3  41.6  90.0   
3    2025        Houston Rockets  82  52  30  0.634  48.2  114.3  42.5  93.4   
4    2025        New York Knicks  82  51  31  0.622  48.5  115.8  43.3  89.2   

   ...  Road_L  E_W  E_L  W_W  W_L  Pre-ASG_W  Pre-ASG_L  Post-ASG_W  \
0  ...       8   29    1   39   13         44         10          24   
1  ...      11   41   11   23    7         44         10          20   
2  ...       8   39   13   22    8         39         16          22   
3  ...      18   21    9   31   21         34         21          18   
4  ...      17   34   18   17   13         36         18          15   

   Post-ASG_L       SOS  
0           4  0.487557  
1           8  0.479675  
2       

## Final Data Cleaning

Now that I have my main dataframes completed, I now need to go through the basic data cleaning checklist:
- checking for NaN values
- checking for duplicate values
- verify manually entered data (Season columns and Team names)

In [264]:
def cleaning_report(df, df_name="DataFrame"):
    print(f"--- Cleaning Report for {df_name} ---")
    
    # 1. Check for NaN values
    nan_summary = df.isna().sum()
    nan_summary = nan_summary[nan_summary > 0]
    if nan_summary.empty:
        print("No NaN values found")
    else:
        print("NaN values detected:")
        print(nan_summary)
    
    # 2. Check for duplicate rows
    dup_count = df.duplicated().sum()
    if dup_count == 0:
        print("No duplicate rows found")
    else:
        print(f"Found {dup_count} duplicate rows")

    # 3. Verify manually entered data (Season & Team)
    if "Season" in df.columns:
        seasons = df["Season"].unique()
        print(f"Unique seasons: {seasons}")
    else:
        print("No 'Season' column found")
        
    if "Team" in df.columns:
        teams = df["Team"].unique()
        print(f"Unique teams: {len(teams)} teams")
        print(sorted(teams))
    else:
        print("No 'Team' column found")
    
    print("-----------------------------------\n")

In [265]:
cleaning_report(front_office_df, "Front Office Data")
cleaning_report(sos_df, "Strength of Schedule")

--- Cleaning Report for Front Office Data ---
NaN values detected:
Payroll    61
dtype: int64
No duplicate rows found
Unique seasons: [2025 2024 2023 2022 2021 2020 2019 2018 2017 2016]
Unique teams: 30 teams
['Atlanta Hawks', 'Boston Celtics', 'Brooklyn Nets', 'Charlotte Hornets', 'Chicago Bulls', 'Cleveland Cavaliers', 'Dallas Mavericks', 'Denver Nuggets', 'Detroit Pistons', 'Golden State Warriors', 'Houston Rockets', 'Indiana Pacers', 'Los Angeles Clippers', 'Los Angeles Lakers', 'Memphis Grizzlies', 'Miami Heat', 'Milwaukee Bucks', 'Minnesota Timberwolves', 'New Orleans Pelicans', 'New York Knicks', 'Oklahoma City Thunder', 'Orlando Magic', 'Philadelphia 76ers', 'Phoenix Suns', 'Portland Trail Blazers', 'Sacramento Kings', 'San Antonio Spurs', 'Toronto Raptors', 'Utah Jazz', 'Washington Wizards']
-----------------------------------

--- Cleaning Report for Strength of Schedule ---
No NaN values found
No duplicate rows found
Unique seasons: [2025 2024 2023 2022 2021 2020 2019 2018 2

Now that I have finished cleaning dataframes, I will move onto the exploration notebook. In this notebook, I will be exploring the data through summary statistics and visualizations with the core goal of understanding the data as best as possible.